In [1]:
import pandas as pd
import numpy as np

from sklearn.metrics import mean_squared_error

# Загрузка данных

В бейзлайне реализована простейшая модель на эго-графе.

Для каждой пары вершин u и v эго-графа попытаемся найти всех общих "друзей" w. Силой связи между вершинами u и v будем считать средную силу связи между ними и общими друзьями.

Поскольку обучение для такой модели не требуется, будем пользоваться только тестовой выборкой.

In [2]:
%%time

test = pd.read_csv("test.csv")
submission = pd.read_csv("submission.csv")

CPU times: total: 19.2 s
Wall time: 27.2 s


In [3]:
submission.head()

,ego_id,u,v,x1
0,8,0,93,0.000000
1,8,0,143,0.000000
2,8,0,151,1.606742
3,8,1,24,0.026496
4,8,5,4,0.159857


Проверочные данные - недоступны участникам, используются для подсчёта метрики:

In [29]:
control = pd.read_csv("control.csv")

Таблицы control и submission отличаются только последним столбцом x1. В таблице control в этом столбце содержатся истинные значения связей x1.

In [32]:
control[["ego_id", "u", "v"]].equals(submission[["ego_id", "u", "v"]])

True

Таблица submission отсортирована по возрастанию ego_id, u, v:

In [4]:
submission.equals(submission.sort_values(["ego_id", "u", "v"]))

True

In [6]:
test.equals(test.sort_values(["ego_id", "u", "v"]))

False

# Модель

In [7]:
submission["ego_id"].value_counts()

ego_id
901943132599     349
721554506143     328
1400159338751    298
1039382085802    270
575525618423     262
                ... 
1142461301166      1
515396076193       1
515396076373       1
51539608193        1
1245540516719      1
Name: count, Length: 20586, dtype: int64

In [8]:
test["ego_id"].value_counts()

ego_id
901943132599     17432
721554506143     16412
1400159338751    14881
1039382085802    13487
575525618423     13081
                 ...  
936302871286        22
1443109012321       20
1529008358147       20
283467841974        19
1503238554422       12
Name: count, Length: 20596, dtype: int64

Константное предсказание:

In [9]:
%%time

submission_dummy = submission.copy()
submission_dummy["x1"] = np.nanmean(test["x1"].values)

CPU times: total: 359 ms
Wall time: 992 ms


In [9]:
%%time

from tqdm import tqdm


ego_id_list = submission["ego_id"].drop_duplicates().values
for ego_id in tqdm(ego_id_list):
    submission_ego_net = submission[submission["ego_id"] == ego_id]
    test_ego_net = test[test["ego_id"] == ego_id]
    friendship = np.zeros_like(submission_ego_net["x1"].values)
    for i, (u, v) in enumerate(zip(submission_ego_net["u"], submission_ego_net["v"])):
        
        u_x1 = test_ego_net.loc[test_ego_net["u"] == u, ["v", "x1", "x2", "x3"]].dropna()
        v_x1 = test_ego_net.loc[test_ego_net["u"] == v, ["v", "x1", "x2", "x3"]].dropna()
        
        common_friends = u_x1.merge(v_x1, on="v")
        
        if common_friends.shape[0] > 0:
            friendship[i] = np.mean(common_friends.drop("v", axis=1).values)
    submission.loc[submission["ego_id"] == ego_id, "x1"] = friendship

  0%|          | 13/20586 [00:02<1:14:52,  4.58it/s]


KeyboardInterrupt: 

# Посчёт метрик

In [51]:
def RMSE(y_true, y_pred):
    return mean_squared_error(y_true, y_pred, squared=False)

In [54]:
print("Dummy model RMSE: {}".format(RMSE(control["x1"], submission_dummy["x1"])))

Dummy model RMSE: 1.3604058861047796


In [63]:
print("Baseline model RMSE: {}".format(RMSE(control["x1"], submission["x1"])))

Baseline model RMSE: 1.353040933001075
